In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing


In [ ]:
colab_path = '/content/drive/MyDrive/ColabNotebooks/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/'

In [ ]:
train = pd.read_csv(colab_path+'/train.csv')
test = pd.read_csv(colab_path+'/test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871393 entries, 0 to 871392
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   871393 non-null  object 
 1   User-ID              871393 non-null  object 
 2   Book-ID              871393 non-null  object 
 3   Book-Rating          871393 non-null  int64  
 4   Age                  871393 non-null  int64  
 5   Location             871393 non-null  object 
 6   Book-Title           871393 non-null  object 
 7   Book-Author          871393 non-null  object 
 8   Year-Of-Publication  871393 non-null  float64
 9   Publisher            871393 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 66.5+ MB


In [ ]:
duplicate = train.duplicated()
print(duplicate.sum())


0


In [ ]:
#train.drop_duplicates(inplace=True)

In [ ]:
# retrieving age data between 5 to 90
train.loc[(train['Age'] > 90) | (train['Age'] < 5), 'Age'] = np.nan
test.loc[(test['Age'] > 90) | (test['Age'] < 5), 'Age'] = np.nan


In [ ]:
## Age 열의 결측값을 최빈값으로 채우기
#train['Age'].fillna(train['Age'].mode()[0], inplace=True)
#test['Age'].fillna(test['Age'].mode()[0], inplace=True)


In [ ]:
train['Age'] = train['Age'].astype(int)
test['Age'] = test['Age'].astype(int)
train['Book-Rating'] = train['Book-Rating'].astype(int)

In [ ]:
train['Book-Rating'].value_counts()

0     548804
8      76971
10     60024
7      55852
9      50494
5      38416
6      26670
4       6462
3       4374
2       2019
1       1307
Name: Book-Rating, dtype: int64

In [ ]:
train['Location'].value_counts().head(10)

toronto, ontario, canada         12267
n/a, n/a, n/a                    11161
chicago, illinois, usa            7214
seattle, washington, usa          6967
ottawa, ontario, canada           6915
olympia, washington, usa          6723
little canada, minnesota, usa     6458
st. louis, missouri, usa          6385
ft. stewart, georgia, usa         5819
san diego, california, usa        5557
Name: Location, dtype: int64

In [ ]:
### lets create a function so that I can easily do it for all features
def Random_value_imputation(feature):
    random_sample=train[feature].dropna().sample(train[feature].isnull().sum())
    random_sample.index=train[train[feature].isnull()].index
    train.loc[train[feature].isnull(),feature]=random_sample

In [ ]:
### lets create a function so that I can easily do it for all features
def Random_value_imputation2(feature):
    random_sample=test[feature].dropna().sample(test[feature].isnull().sum())
    random_sample.index=test[test[feature].isnull()].index
    test.loc[test[feature].isnull(),feature]=random_sample

In [ ]:
for col in train:
    Random_value_imputation(col)

In [ ]:
for col in test:
    Random_value_imputation2(col)

In [ ]:
test.isnull().sum()

ID                     0
User-ID                0
Book-ID                0
Age                    0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Country                0
dtype: int64

In [ ]:
num_features=[col for col in train.columns if train[col].dtype!='O']
num_features

['Book-Rating', 'Age', 'Year-Of-Publication']

In [ ]:
num_data = train[num_features]
num_data.head()

,Book-Rating,Age,Year-Of-Publication
0,8,23,2001.0
1,8,23,1981.0
2,0,23,1981.0
3,0,23,1991.0
4,8,23,1989.0


In [ ]:
cat_features=[col for col in train.columns if train[col].dtype=='O']
cat_features

['ID',
 'User-ID',
 'Book-ID',
 'Location',
 'Book-Title',
 'Book-Author',
 'Publisher']

In [ ]:
categorical=train[cat_features]
categorical.head()

,ID,User-ID,Book-ID,Location,Book-Title,Book-Author,Publisher
0,TRAIN_000000,USER_00000,BOOK_044368,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,Mira
1,TRAIN_000001,USER_00000,BOOK_081205,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,Penguin Books
2,TRAIN_000002,USER_00000,BOOK_086781,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,Penguin Books
3,TRAIN_000003,USER_00000,BOOK_098622,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,Avon
4,TRAIN_000004,USER_00000,BOOK_180810,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,Signet Book


In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.10)
    quartile3 = dataframe[variable].quantile(0.90)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
def has_outliers(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    if dataframe[(dataframe[variable] < low_limit) | (dataframe[variable] > up_limit)].any(axis=None):
        print(variable, "yes")
    print(variable, "no")

In [ ]:
for col in num_data.columns:
    has_outliers(num_data, col)

Book-Rating no
Age no
Year-Of-Publication yes
Year-Of-Publication no


In [ ]:
plt.figure(figsize=(22,18))
for i,col in enumerate(num_data.columns):
    plt.subplot(4,9,i+1)
    sns.boxplot(num_data[col])

In [ ]:
## function to get ride of qutliers
def replace_with_thresholds(dataframe, numeric_columns):
    for variable in numeric_columns:
        low_limit, up_limit = outlier_thresholds(dataframe, variable)
        dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
        dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
replace_with_thresholds(num_data, num_data.columns)

In [ ]:
for col in num_data.columns:
    has_outliers(num_data, col)

Book-Rating no
Age no
Year-Of-Publication no


In [ ]:
plt.figure(figsize=(22,18))
for i,col in enumerate(num_data.columns):
    plt.subplot(4,9,i+1)
    sns.boxplot(num_data[col])

In [ ]:
categorical['Book-Author'].value_counts()

Stephen King           8467
Nora Roberts           6934
John Grisham           5283
James Patterson        5020
Mary Higgins Clark     3983
                       ... 
Cyrinda Foxe-Tyler        1
Matt Bloom                1
Cassie Kendall            1
Birney, M.D. Dibble       1
Stephen Griffith          1
Name: Book-Author, Length: 92635, dtype: int64

In [ ]:
from wordcloud import WordCloud
plt.subplots(figsize=(25,15))
wordcloud = WordCloud(
                          background_color='black',
                          width=1920,
                          height=1080
                         ).generate(" ".join(train['Book-Author']))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
train["Book-Rating"].value_counts().plot.pie(autopct = "%1.1f%%")

In [ ]:
import plotly.express as px

def violin(col):
    fig = px.violin(train, y=col, x="Book-Rating", color="Book-Rating", box=True , template = 'plotly_dark')
    return fig.show()

def scatters(col1,col2):
    fig = px.scatter(train, x=col1, y=col2, color="Book-Rating" ,template = 'plotly_dark')
    return fig.show()

def kde_plot(feature):
    grid = sns.FacetGrid(train, hue="Book-Rating",aspect=2)
    grid.map(sns.kdeplot, feature)
    grid.add_legend()

In [ ]:
kde_plot('Age')

In [ ]:
scatters('Year-Of-Publication', 'Age')

In [ ]:
merged_df = pd.concat([num_data, categorical], axis=1)

In [ ]:
train = train[(train['Year-Of-Publication'] >= 1950) | (train['Year-Of-Publication'] == -1)]

In [ ]:
def extract_country(location):
    # 구분자로 분할하여 마지막 요소 추출
    parts = location.split(',')
    country = parts[-1].strip()  # 공백 제거
    return country

# 'Location' 열에서 나라 정보 추출
train['Country'] = train['Location'].apply(extract_country)
test['Country'] = test['Location'].apply(extract_country)


In [ ]:
#train = train.drop('Location', axis=1)
#test = test.drop('Location', axis=1)


In [ ]:
train['Country'] = train['Country'].replace('', np.nan)
test['Country'] = test['Country'].replace('', np.nan)


In [ ]:
train['Country'].value_counts()

usa                 630043
canada               78571
united kingdom       27050
germany              23471
australia            15270
                     ...  
palau                    1
galiza neghra            1
macedonia                1
prince william           1
richmond country         1
Name: Country, Length: 243, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 책의 제목과 저자를 결합한 새로운 컬럼 생성
train['title_author'] = train['Book-Title'] + ' by ' + train['Book-Author']
test['title_author'] = test['Book-Title'] + ' by ' + test['Book-Author']

In [ ]:
#사용자가 평점을 매긴 총 아이템 수
train['Total_Items_Rated_By_User'] = train.groupby('User-ID')['Book-Rating'].transform('count')

In [ ]:
#특정 아이템에 대해 매긴 평점과 해당 아이템의 평균 평점의 차이:
train['Item_Rating_Difference'] = train.groupby('Book-ID')['Book-Rating'].transform(lambda x: x - x.mean())

In [ ]:
#사용자의 평균 평점에 비해 해당 아이템의 평점 차이:
train['User_Item_Rating_Difference'] = train.groupby('User-ID')['Book-Rating'].transform(lambda x: x - x.mean())

In [ ]:
train

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher,Country,title_author,Total_Items_Rated_By_User,Item_Rating_Difference,User_Item_Rating_Difference
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira,canada,Road Taken by Rona Jaffe,8,5.571429,3.25
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books,canada,Macbeth (New Penguin Shakespeare) by William S...,8,4.000000,3.25
2,TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",Waverley (Penguin English Library),Walter Scott,1981.0,Penguin Books,canada,Waverley (Penguin English Library) by Walter S...,8,0.000000,-4.75
3,TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",Mother Earth Father Sky,Sue Harrison,1991.0,Avon,canada,Mother Earth Father Sky by Sue Harrison,8,-3.086957,-4.75
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book,canada,She Who Remembers by Linda Lay Shuler,8,6.301887,3.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871388,TRAIN_871388,USER_92096,BOOK_081138,0,34.0,"minneapolis, minnesota, usa",Healing Words: The Power of Prayer and the Pra...,Larry Dossey,1993.0,Harpercollins,usa,Healing Words: The Power of Prayer and the Pra...,1,0.000000,0.00
871389,TRAIN_871389,USER_92097,BOOK_258124,0,35.0,"temple, texas, usa",The Salmon of Doubt: Hitchhiking the Galaxy On...,DOUGLAS ADAMS,2002.0,Harmony,usa,The Salmon of Doubt: Hitchhiking the Galaxy On...,1,-3.571429,0.00
871390,TRAIN_871390,USER_92098,BOOK_071848,0,45.0,"ottawa, ontario, canada",Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2000.0,Thorndike Press,canada,Harry Potter and the Prisoner of Azkaban (Book...,1,-4.600000,0.00
871391,TRAIN_871391,USER_92099,BOOK_252599,8,43.0,"maple grove, minnesota, usa",Heartbreak Hill: Anatomy of a Ryder Cup,Tim Rosaforte,1996.0,St Martins Pr,usa,Heartbreak Hill: Anatomy of a Ryder Cup by Tim...,1,4.000000,0.00


In [ ]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095479 sha256=7a3022dfbc4750be7e92cc83cd3f4dd70ca8af2ab17c1496cb0916cd7a872ad7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD, Dataset, Reader, accuracy, model_selection
# Surprise 라이브러리용 Reader 및 Dataset 객체 생성
reader = Reader(rating_scale=(0, 10))
train1 = Dataset.load_from_df(train[['User-ID', 'Book-ID', 'Book-Rating']], reader)
train2 = train1.build_full_trainset()

In [ ]:
# SVD 모델 훈련
model = SVD()
model.fit(train2)

In [ ]:
# Train on books dataset
%time model_selection.cross_validate(model, train1, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.5044  3.4987  3.5056  3.5017  3.4989  3.5019  0.0028  
Fit time          21.21   22.13   22.37   22.00   21.67   21.87   0.40    
Test time         1.85    3.11    2.95    3.03    1.63    2.51    0.64    
CPU times: user 2min 18s, sys: 2.02 s, total: 2min 20s
Wall time: 2min 22s


{'test_rmse': array([3.50444536, 3.49865853, 3.50564331, 3.50168514, 3.49892169]),
 'fit_time': (21.209107875823975,
  22.125531673431396,
  22.366341590881348,
  21.99923300743103,
  21.66634726524353),
 'test_time': (1.8531122207641602,
  3.1054201126098633,
  2.9483025074005127,
  3.025731325149536,
  1.6261532306671143)}

In [ ]:
submit  = pd.read_csv(colab_path + '/sample_submission.csv')


In [ ]:
submit['Book-Rating'] = test.apply(lambda row: model.predict(row['User-ID'], row['Book-ID']).est, axis=1)

In [ ]:
submit['Book-Rating'] = submit['Book-Rating'].round()
submit['Book-Rating']

0         5.0
1         7.0
2         5.0
3         7.0
4         5.0
         ... 
159616    4.0
159617    3.0
159618    3.0
159619    3.0
159620    3.0
Name: Book-Rating, Length: 159621, dtype: float64

In [ ]:
submit.to_csv('./svd35.csv', index=False)